# SQL For Data Analysis


## The TMDb database

In this project I explore the TMDB database – an online movie and TV show database that houses some of the most popular movies and TV shows.

Below is an Entity Relationship Diagram (ERD) of the TMDb database:

<img src="https://github.com/Explore-AI/Pictures/blob/master/TMDB_ER_diagram.png?raw=true" width=70%/>

As can be seen from the ERD, the TMDb database consists of `12 tables` containing information about movies, cast, genre, and so much more.

## Loading the database

In [1]:
# Load and activate the SQL extension to allow us to execute SQL in a Jupyter notebook. 
%load_ext sql

Next, go ahead and load your database. To do this, you will need to ensure you have downloaded the `TMDB.db` sqlite file from Athena and have stored it in a known location.

In [2]:
# Establish a connection to the local database using the '%sql' magic command.
%sql sqlite:///TMDB.db

'Connected: @TMDB.db'

## QUESTIONS

### 1.Who won the Oscar for “Actor in a Leading Role” in  2015?
(The `winner is indicated as '1.0'`.)

In [4]:
%%sql

SELECT 
    winner,name,film,year
FROM 
    oscars 
WHERE 
    award = 'Actor in a Leading Role' 
    AND year = 2015 AND winner = 1.0


 * sqlite:///TMDB.db
Done.


winner,name,film,year
1.0,Leonardo DiCaprio,The Revenant,2015


### 2.Extract the ten oldest movies in the database?

In [5]:
%%sql

SELECT 
    title AS Movie,
    SUBSTR(release_date, 1,11) AS "Date Released"
FROM movies
WHERE release_date IS NOT NULL
ORDER BY release_date ASC
LIMIT 10;

 * sqlite:///TMDB.db
Done.


Movie,Date Released
Intolerance,1916-09-04
The Big Parade,1925-11-05
Metropolis,1927-01-10
Pandora's Box,1929-01-30
The Broadway Melody,1929-02-08
Hell's Angels,1930-11-15
A Farewell to Arms,1932-12-08
42nd Street,1933-02-02
She Done Him Wrong,1933-02-09
It Happened One Night,1934-02-22


### 3.How many movies are there that are both in the `Thriller` genre and contain the word `love` anywhere in the keywords?

In [7]:
%%sql

SELECT COUNT(DISTINCT m.title) AS "Number of Movies" 
FROM movies AS m
JOIN keywordmap kmp 
ON kmp.movie_id = m.movie_id
JOIN keywords k 
ON k.keyword_id = kmp.keyword_id
JOIN genremap gmp 
ON gmp.movie_id = m.movie_id
JOIN genres g 
ON gmp.genre_id = g.genre_id
WHERE g.genre_name = "Thriller" 
      AND k.keyword_name LIKE "%love%";


 * sqlite:///TMDB.db
Done.


Number of Movies
48


### 4. How many movies are there that were released between 1 August 2006 ('2006-08-01') and 1 October 2009 ('2009-10-01') that have a popularity score of more than 40 and a budget of less than 50 000 000?

In [8]:
%%sql

SELECT COUNT(*) AS "Number of Movies"
FROM movies 
WHERE release_date > '2006-08-01' AND release_date < '2009-10-01' 
      AND popularity > 40 AND budget < 50000000;


 * sqlite:///TMDB.db
Done.


Number of Movies
29


### 5.What are the three production companies that have the highest movie popularity score on average, as recorded within the database?

In [20]:
%%sql

SELECT 
    pcs.production_company_name AS "Production Company",
    ROUND(AVG(m.popularity) ,1) AS Popularity
FROM
    movies m
JOIN
    productioncompanymap pmp ON m.movie_id = pmp.movie_id
JOIN
    productioncompanies pcs ON pcs.production_company_id = pmp.production_company_id
GROUP BY
    pcs.production_company_name
ORDER BY
    Popularity DESC
LIMIT 3;


 * sqlite:///TMDB.db
Done.


Production Company,Popularity
The Donners' Company,514.6
Bulletproof Cupid,481.1
Kinberg Genre,326.9


### 6.Which genre has, on average, the lowest movie popularity score?

In [17]:
%%sql

SELECT 
    g.genre_name AS Genre,
    ROUND(AVG(m.popularity), 4) AS Popularity
FROM    
    movies m
JOIN  
    genremap gmp ON m.movie_id = gmp.movie_id
JOIN 
    genres g ON g.genre_id = gmp.genre_id
GROUP BY 
    g.genre_name
ORDER BY Popularity ASC
LIMIT 1;

 * sqlite:///TMDB.db
Done.


Genre,Popularity
Foreign,0.6868


### 7.DStv will be having a special week dedicated to the actor Alan Rickman. Create  new _view_ that shows the titles, release dates, taglines, and overviews of all movies that Alan Rickman has played in.

In [24]:
%%sql

CREATE VIEW Alan_Rickman_Movies AS
SELECT 
    m.title AS Movie,
    m.release_date AS "Date",
    m.tagline AS Tagline, 
    m.overview AS Overview
FROM 
    movies m
JOIN 
    casts c ON c.movie_id = m.movie_id
JOIN 
    actors a ON a.actor_id = c.actor_id
WHERE 
    a.actor_name = 'Alan Rickman';

 * sqlite:///TMDB.db
Done.


[]

### 8. Which three actors have appeared in the most movies?

In [27]:
%%sql

WITH Ranked_actors AS
(   
    SELECT 
        actor_name, 
        COUNT(movies.title) AS movie_count,
        ROW_NUMBER() OVER (ORDER BY COUNT(movies.title) DESC) AS rank
    FROM 
        actors
    LEFT JOIN 
        casts ON actors.actor_id = casts.actor_id
    LEFT JOIN 
        movies ON movies.movie_id = casts.movie_id
    GROUP BY 
        actor_name
)
SELECT 
    rank AS "Pos.",
    actor_name AS Actor,
    movie_count AS "No. of movies"    
FROM 
    Ranked_actors
WHERE 
    rank <= 3


 * sqlite:///TMDB.db
Done.


Pos.,Actor,No. of movies
1,Samuel L. Jackson,67
2,Robert De Niro,57
3,Bruce Willis,51


### 9. Calculate the total revenue generated for each year between 2010 and 2015?

In [29]:
%%sql

SELECT 
    strftime('%Y', release_date) AS Year,
    SUM(revenue) AS "Total Revenue"
FROM movies
WHERE strftime('%Y', release_date) BETWEEN '2010' AND '2015'
GROUP BY Year;

 * sqlite:///TMDB.db
Done.


Year,Total Revenue
2010,20348574768.0
2011,20516921160.0
2012,24141710246.0
2013,23411493295.0
2014,24120490589.0
2015,22775024221.0


### 10.What is the average runtime of movies in each genre?

In [30]:
%%sql
SELECT 
    g.genre_name AS Genre, 
    ROUND(AVG(runtime), 2) AS Average_runtime
FROM 
    genres g
JOIN 
    genremap gmp ON gmp.genre_id = g.genre_id
JOIN 
    movies m ON gmp.movie_id = m.movie_id 
GROUP BY 
    genre_name
ORDER BY 
    Average_runtime;


 * sqlite:///TMDB.db
Done.


Genre,Average_runtime
TV Movie,85.63
Animation,89.92
Documentary,93.82
Horror,95.95
Family,97.3
Comedy,100.03
Fantasy,107.28
Science Fiction,107.48
Thriller,107.54
Romance,109.38
